In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
nba_2024_season_schedule = pd.read_csv('C:/Users/LukeB/Downloads/NBA-Full-Schedule-frills-2023-2024.csv')

filtered_schedule = nba_2024_season_schedule[
    (nba_2024_season_schedule['home_team'].isin(['NYK', 'MEM'])) |
    (nba_2024_season_schedule['away_team'].isin(['NYK', 'MEM']))
]

filtered_schedule = filtered_schedule.reset_index(drop=True)

filtered_schedule.info()

In [3]:
filtered_schedule['game_date'] = pd.to_datetime(filtered_schedule['game_date'])

In [4]:
import praw
import re

CLIENT_ID = "XFQo87DDjus70pAmVFCi-g"
CLIENT_SECRET = "gZnURWimF9GKa191vjS0jaA4FYaG4w"
USER_AGENT = "SubstantialAd2572"

reddit = praw.Reddit(
    client_id = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    user_agent = USER_AGENT
)

In [ ]:
urls_Memphis = []
for thread in reddit.subreddit('memphisgrizzlies').new(limit=1000):
    url = thread.url
    if re.search(r'www.reddit.com/r/' , url) and '/comments/' in url:
        urls_Memphis.append(url)

In [ ]:
urls_NY = []
for thread in reddit.subreddit('NYKnicks').new(limit=1000):
    url = thread.url
    if re.search(r'www.reddit.com/r/' , url) and '/comments/' in url:
        urls_NY.append(url)

In [ ]:
urls_all = urls_Memphis + urls_NY

In [ ]:
import time

created_times = []
post_urls = []
m = 0

for url in urls_all:
    submission = reddit.submission(url=url)
    created_times.append(submission.created_utc)
    post_urls.append(url)
    time.sleep(3)

date_link_df = pd.DataFrame({'URL': post_urls}, index=pd.to_datetime(created_times, unit='s'))

date_link_df.head()

In [ ]:
date_link_df.to_csv('C:/Users/LukeB/OneDrive/Documents/Datasets/links_data_full.csv')

In [ ]:
urls_df = pd.read_csv('C:/Users/LukeB/OneDrive/Documents/Datasets/links_data_full.csv')
urls_df

In [ ]:
urls_df = urls_df.rename(columns={'Unnamed: 0': 'date'})
urls_df = urls_df.sort_values(by='date')
urls_df = urls_df.drop(200)
urls_df.reset_index(drop=True, inplace=True)
urls_df['date'] = pd.to_datetime(urls_df['date'])
urls_df['date'] = urls_df['date'].dt.date
urls_df.info()

In [ ]:
filtered_schedule_fin = filtered_schedule.loc[:, ['game_date', 'away_team', 'home_team']]
filtered_schedule_fin.info()

In [ ]:
filtered_schedule_fin['game_date'] = pd.to_datetime(filtered_schedule_fin['game_date'])

urls_df['date'] = pd.to_datetime(urls_df['date'])

Game_date_Grizz = filtered_schedule_fin[((filtered_schedule_fin['away_team'] == 'MEM') | (filtered_schedule_fin['home_team'] == 'MEM'))]
Game_date_Knicks = filtered_schedule_fin[((filtered_schedule_fin['away_team'] == 'NYK') | (filtered_schedule_fin['home_team'] == 'NYK'))]




Gameday_Grizz = urls_df[urls_df['date'].isin(Game_date_Grizz['game_date']) & urls_df['URL'].str.contains('memphisgrizzlies')]
Not_Gameday_Grizz = urls_df[~urls_df.index.isin(Gameday_Grizz.index) & urls_df['URL'].str.contains('memphisgrizzlies')]

Gameday_Knicks = urls_df[urls_df['date'].isin(Game_date_Knicks['game_date']) & urls_df['URL'].str.contains('NYKnicks')]
Not_Gameday_Knicks = urls_df[~urls_df.index.isin(Gameday_Knicks.index) & urls_df['URL'].str.contains('NYKnicks')]


In [ ]:
from urllib.request import urlopen
import json

In [ ]:
link_texts = []
exception_urls = []

for url in Gameday_Grizz['URL']:
    reddit_text = ""
    
    try:
        response = urlopen(url + '.json')
        json_data = json.load(response)
        
        for item in json_data:
            if 'data' in item and 'children' in item['data']:
                for child in item['data']['children']:
                    if 'data' in child and 'title' in child['data'] and 'selftext' in child['data']:
                        reddit_text += child['data']['title'] + "\n" + child['data']['selftext'] + "\n"
                    elif 'data' in child and 'body' in child['data']:
                        reddit_text += child['data']['body'] + "\n"
        
        link_texts.append(reddit_text)
    except Exception as e:
        print("Error fetching data for URL:", url, "-", e)
        exception_urls.append(url)
    
    time.sleep(3)

In [ ]:
Text_Gameday_Grizz = Gameday_Grizz[~Gameday_Grizz['URL'].isin(exception_urls)].copy()
boolean_list = [True] * 224
Text_Gameday_Grizz.loc[:, 'text'] = link_texts
Text_Gameday_Grizz['game'] = boolean_list
Text_Gameday_Grizz = Text_Gameday_Grizz.drop(columns = ['URL'])
Text_Gameday_Grizz

In [ ]:
link_texts1 = []
exception_urls1 = []

for url in Not_Gameday_Grizz['URL']:
    reddit_text = ""
    
    try:
        response = urlopen(url + '.json')
        json_data = json.load(response)
        
        for item in json_data:
            if 'data' in item and 'children' in item['data']:
                for child in item['data']['children']:
                    if 'data' in child and 'title' in child['data'] and 'selftext' in child['data']:
                        reddit_text += child['data']['title'] + "\n" + child['data']['selftext'] + "\n"
                    elif 'data' in child and 'body' in child['data']:
                        reddit_text += child['data']['body'] + "\n"
        
        link_texts1.append(reddit_text)
    except Exception as e:
        print("Error fetching data for URL:", url, "-", e)
        exception_urls1.append(url)
    
    time.sleep(3)

In [ ]:
Text_Not_Gameday_Grizz = Not_Gameday_Grizz[~Not_Gameday_Grizz['URL'].isin(exception_urls1)].copy()
boolean_list1 = [False] * 247
Text_Not_Gameday_Grizz.loc[:, 'text'] = link_texts1
Text_Not_Gameday_Grizz['game'] = boolean_list1
Text_Not_Gameday_Grizz = Text_Not_Gameday_Grizz.drop(columns = ['URL'])
Text_Not_Gameday_Grizz

In [ ]:
Grizzlies_Text_DF = pd.concat([Text_Gameday_Grizz, Text_Not_Gameday_Grizz])
Grizzlies_Text_DF = Grizzlies_Text_DF.sort_values(by='date')
Grizzlies_Text_DF.reset_index(drop=True, inplace=True)
Grizzlies_Text_DF#Exporting this DF to csv

In [ ]:
link_texts2 = []
exception_urls2 = []

for url in Gameday_Knicks['URL']:
    reddit_text = ""
    
    try:
        response = urlopen(url + '.json')
        json_data = json.load(response)
        
        for item in json_data:
            if 'data' in item and 'children' in item['data']:
                for child in item['data']['children']:
                    if 'data' in child and 'title' in child['data'] and 'selftext' in child['data']:
                        reddit_text += child['data']['title'] + "\n" + child['data']['selftext'] + "\n"
                    elif 'data' in child and 'body' in child['data']:
                        reddit_text += child['data']['body'] + "\n"
        
        link_texts2.append(reddit_text)
    except Exception as e:
        print("Error fetching data for URL:", url, "-", e)
        exception_urls2.append(url)
    
    time.sleep(3)

In [ ]:
Text_Gameday_Knicks = Gameday_Knicks[~Gameday_Knicks['URL'].isin(exception_urls2)].copy()
boolean_list2 = [True] * l
Text_Gameday_Knicks.loc[:, 'text'] = link_texts2
Text_Gameday_Knicks['game'] = boolean_list2
Text_Gameday_Knicks = Text_Gameday_Knicks.drop(columns = ['URL'])
Text_Gameday_Knicks

In [ ]:
link_texts3 = []
exception_urls3 = []

for url in Not_Gameday_Knicks['URL']:
    reddit_text = ""
    
    try:
        response = urlopen(url + '.json')
        json_data = json.load(response)
        
        for item in json_data:
            if 'data' in item and 'children' in item['data']:
                for child in item['data']['children']:
                    if 'data' in child and 'title' in child['data'] and 'selftext' in child['data']:
                        reddit_text += child['data']['title'] + "\n" + child['data']['selftext'] + "\n"
                    elif 'data' in child and 'body' in child['data']:
                        reddit_text += child['data']['body'] + "\n"
        
        link_texts3.append(reddit_text)
    except Exception as e:
        print("Error fetching data for URL:", url, "-", e)
        exception_urls3.append(url)
    
    time.sleep(3)

In [ ]:
Text_Not_Gameday_Knicks = Not_Gameday_Knicks[~Not_Gameday_Knicks['URL'].isin(exception_urls3)].copy()
boolean_list3 = [False] * l1
Text_Not_Gameday_Knicks.loc[:, 'text'] = link_texts3
Text_Not_Gameday_Knicks['game'] = boolean_list3
Text_Not_Gameday_Knicks = Text_Not_Gameday_Knicks.drop(columns = ['URL'])
Text_Not_Gameday_Knicks

In [ ]:
Knicks_Text_DF = pd.concat([Text_Gameday_Knicks, Text_Not_Gameday_Knicks])
Knicks_Text_DF = Knicks_Text_DF.sort_values(by='date')
Knicks_Text_DF.reset_index(drop=True, inplace=True)
Knicks_Text_DF#Exporting this DF to csv

In [ ]:
Knicks_Text_DF.to_csv('C:/Users/LukeB/OneDrive/Documents/Datasets/Knicks_Text_Data.csv', index = False)
Grizzlies_Text_DF.to_csv('C:/Users/LukeB/OneDrive/Documents/Datasets/Grizzlies_Text_data.csv', index = False)

In [ ]:
Grizzlies = pd.read_csv('C:/Users/LukeB/OneDrive/Documents/Datasets/Grizzlies_Text_data.csv')
Knicks = pd.read_csv('C:/Users/LukeB/OneDrive/Documents/Datasets/Knicks_Text_Data.csv')